<a href="https://colab.research.google.com/github/nagendra405/ML/blob/master/NLP_with_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from nltk import word_tokenize, sent_tokenize
from gensim.models.word2vec import Word2Vec
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, SpatialDropout1D, Conv1D, GlobalMaxPooling1D, MaxPooling1D, LSTM
from keras.layers import Embedding
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.layers.normalization import BatchNormalization
from keras.layers import SimpleRNN
from keras.layers.wrappers import Bidirectional

Using TensorFlow backend.


In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1uo_H7HCEkXsQjjzrkTJ8E_Ns7pm6YKwE'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('df.csv')

    100% |████████████████████████████████| 993kB 20.3MB/s 


In [0]:
reviews_df = pd.read_csv("df.csv")
reviews_df["review"] = reviews_df["Negative_Review"] + reviews_df["Positive_Review"]
reviews_df["is_bad_review"] = reviews_df["Reviewer_Score"].apply(lambda x: 1 if x < 5 else 0)
reviews_df = reviews_df[["review", "is_bad_review"]]
reviews_df["review"] = reviews_df["review"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))

In [4]:
reviews_df['is_bad_review'].value_counts()

0    493457
1     22281
Name: is_bad_review, dtype: int64

In [0]:
reviews_df_negs = reviews_df[reviews_df['is_bad_review']==1]
reviews_df_pos = reviews_df[reviews_df['is_bad_review']==0].sample(n=reviews_df_negs.shape[0],replace=False)

In [6]:
print(reviews_df_negs.shape)
print(reviews_df_pos.shape)

(22281, 2)
(22281, 2)


In [0]:
rev_df = pd.concat((reviews_df_negs,reviews_df_pos))
np.random.seed(42)
rev_df = rev_df.reindex(np.random.permutation(rev_df.index))
rev_df.reset_index(inplace=True, drop=True)

In [0]:
test_data=rev_df.sample(frac=0.1,random_state=42)
train_data=rev_df.drop(test_data.index)

In [9]:
print(train_data.shape,test_data.shape)
print(train_data.isnull().sum())
print(test_data.isnull().sum())

(40106, 2) (4456, 2)
review           0
is_bad_review    0
dtype: int64
review           0
is_bad_review    0
dtype: int64


In [0]:
val_data=train_data.sample(frac=0.1,random_state=42)
train_data=train_data.drop(val_data.index)

In [11]:
print(train_data.shape,val_data.shape)
print(train_data.isnull().sum())
print(val_data.isnull().sum())

(36095, 2) (4011, 2)
review           0
is_bad_review    0
dtype: int64
review           0
is_bad_review    0
dtype: int64


In [0]:
NUM_WORDS=20000

In [13]:
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',lower=True)
tokenizer.fit_on_texts(train_data['review'])
sequences_train = tokenizer.texts_to_sequences(train_data['review'])
sequences_valid=tokenizer.texts_to_sequences(val_data['review'])
sequences_test=tokenizer.texts_to_sequences(test_data['review'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 24992 unique tokens.


In [14]:
X_train = pad_sequences(sequences_train)
X_val = pad_sequences(sequences_valid,maxlen=X_train.shape[1])
y_train = np.asarray(train_data['is_bad_review'])
y_val = np.asarray(val_data['is_bad_review'])
X_test = pad_sequences(sequences_test,maxlen=X_train.shape[1])
y_test = np.asarray(test_data['is_bad_review'])
print('Shape of X train and X validation tensor:', X_train.shape,X_val.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_val.shape)
print('Shape of X_test and y_test:', X_test.shape,y_test.shape)

Shape of X train and X validation tensor: (36095, 689) (4011, 689)
Shape of label train and validation tensor: (36095,) (4011,)
Shape of X_test and y_test: (4456, 689) (4456,)


In [15]:
test_data['is_bad_review'].value_counts()

0    2278
1    2178
Name: is_bad_review, dtype: int64

In [0]:
def model_evolution(model,X_test,y_test,threshold) :
  y_hat = model.predict_proba(X_test)
  y_hat = [y_hat > threshold]
  y_hat = y_hat[0].reshape(y_test.shape)
  print(confusion_matrix(y_test,y_hat))
  print(classification_report(y_test,y_hat))

In [0]:
# training:
epochs = 10
batch_size = 128
n_dim = 256
max_input_length = X_train.shape[1] # can be tuned at pad sequence and embedding leyer

#Evolution
threshold = 0.5

In [0]:
def model_evalute(X, Y) :
  # load model
  model_ann_one = load_model(X)
  # load weights
  model_ann_one.load_weights(Y)

  # Model Evolution
  model_evolution(model_ann_one,X_test,y_test,threshold)

In [19]:
# Shallow network- ANN

def ANN_one_hidden(NUM_WORDS,n_dim,max_input_length,X_train, y_train,batch_size,epochs,X_val, y_val) :
  
  model = Sequential()
  model.add(Embedding(NUM_WORDS,n_dim, input_length=max_input_length))
  model.add(Flatten())
  
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  
  model.add(Dense(1, activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  weight_file="ann.one.weights.best.hdf5"
  model_name ="ann.one.h5"
  
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(weight_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks_list)
  
  model.save(model_name)
  
  return model_name,weight_file

# model fitting and saving into drive

x, y = ANN_one_hidden(NUM_WORDS,n_dim,max_input_length,X_train, y_train,batch_size,epochs,X_val, y_val)
model_evalute(x, y)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 36095 samples, validate on 4011 samples
Epoch 1/10
36095/36095 [==============================] - 30s 843us/step - loss: 3.0226 - acc: 0.6796 - val_loss: 0.3595 - val_acc: 0.8507

Epoch 00001: val_acc improved from -inf to 0.85066, saving model to ann.one.weights.best.hdf5
Epoch 2/10
36095/36095 [==============================] - 27s 738us/step - loss: 0.2872 - acc: 0.8821 - val_loss: 0.3686 - val_acc: 0.8397

Epoch 00002: val_acc did not improve from 0.85066
Epoch 3/10
36095/36095 [==============================] - 27s 735us/step - loss: 0.1719 - acc: 0.9372 - val_loss: 0.4195 - val_acc: 0.8327

Epoch 00003: val_acc did not improve from 0.85066
[[1927  351]
 [ 300 1878]]
              precision    recall  f1-score   support

           0       0

In [20]:
def ANN_two_hidden(NUM_WORDS,n_dim,max_input_length,X_train, y_train,batch_size,epochs,X_val, y_val) :
  
  model = Sequential()
  model.add(Embedding(NUM_WORDS,n_dim, input_length=max_input_length))
  model.add(Flatten())
  
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  
  model.add(Dense(1, activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  weight_file="ann.two.weights.best.hdf5"
  model_name ="ann.two.h5"
  
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(weight_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks_list)
  
  model.save(model_name)
 
  return model_name,weight_file

# model fitting and saving into drive

x, y = ANN_two_hidden(NUM_WORDS,n_dim,max_input_length,X_train, y_train,batch_size,epochs,X_val, y_val)
model_evalute(x, y)

Train on 36095 samples, validate on 4011 samples
Epoch 1/10
36095/36095 [==============================] - 28s 787us/step - loss: 0.6017 - acc: 0.7591 - val_loss: 0.3449 - val_acc: 0.8519

Epoch 00001: val_acc improved from -inf to 0.85191, saving model to ann.two.weights.best.hdf5
Epoch 2/10
36095/36095 [==============================] - 27s 747us/step - loss: 0.2765 - acc: 0.8890 - val_loss: 0.3614 - val_acc: 0.8489

Epoch 00002: val_acc did not improve from 0.85191
Epoch 3/10
36095/36095 [==============================] - 27s 745us/step - loss: 0.1623 - acc: 0.9398 - val_loss: 0.4660 - val_acc: 0.8355

Epoch 00003: val_acc did not improve from 0.85191
[[1890  388]
 [ 264 1914]]
              precision    recall  f1-score   support

           0       0.88      0.83      0.85      2278
           1       0.83      0.88      0.85      2178

   micro avg       0.85      0.85      0.85      4456
   macro avg       0.85      0.85      0.85      4456
weighted avg       0.85      0.85     

In [21]:
def CNN_one_layer(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val) :
  
  model = Sequential()
  model.add(Embedding(NUM_WORDS,n_dim, input_length=max_input_length))
  model.add(SpatialDropout1D(0.2))
  
  model.add(Conv1D(256, 3, activation='relu'))
  model.add(GlobalMaxPooling1D())
  
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.2))
  
  model.add(Dense(1, activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  weight_file="cnn.one.weights.best.hdf5"
  model_name ="cnn.one.h5"
  
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(weight_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks_list)
  
  model.save(model_name)
  
  return model_name,weight_file

# model fitting and saving into drive

x, y = CNN_one_layer(NUM_WORDS,n_dim,max_input_length,X_train, y_train,batch_size,epochs,X_val, y_val)
model_evalute(x, y)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 36095 samples, validate on 4011 samples
Epoch 1/10
36095/36095 [==============================] - 31s 851us/step - loss: 0.3793 - acc: 0.8278 - val_loss: 0.3260 - val_acc: 0.8651

Epoch 00001: val_acc improved from -inf to 0.86512, saving model to cnn.one.weights.best.hdf5
Epoch 2/10
36095/36095 [==============================] - 27s 758us/step - loss: 0.2548 - acc: 0.8993 - val_loss: 0.3407 - val_acc: 0.8629

Epoch 00002: val_acc did not improve from 0.86512
Epoch 3/10
36095/36095 [==============================] - 27s 760us/step - loss: 0.1700 - acc: 0.9364 - val_loss: 0.3993 - val_acc: 0.8464

Epoch 00003: val_acc did not improve from 0.86512
[[1937  341]
 [ 269 1909]]
              precision    recall  f1-score   support

           0       0.88      0.85      0.86      2278
           1       0.85      0.88      0.86      2178

   micro avg       0.86      0.86      0.86      4456
   macro avg  

In [22]:
def CNN_lenet(NUM_WORDS,n_dim,max_input_length,X_train, y_train,batch_size,epochs,X_val, y_val) :
  
  model = Sequential()
  model.add(Embedding(NUM_WORDS,n_dim, input_length=max_input_length))
  model.add(SpatialDropout1D(0.2))
  
  model.add(Conv1D(256, 3, activation='relu'))
  model.add(Conv1D(256, 3, activation='relu'))
  model.add(GlobalMaxPooling1D())
  
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.2))
  
  model.add(Dense(1, activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  weight_file="cnn.lenet.weights.best.hdf5"
  model_name ="cnn.lenet.h5"
  
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(weight_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks_list)
  
  model.save(model_name)
  
  return model_name,weight_file

# model fitting and saving into drive

x, y = CNN_lenet(NUM_WORDS,n_dim,max_input_length,X_train, y_train,batch_size,epochs,X_val, y_val)
model_evalute(x, y)

Train on 36095 samples, validate on 4011 samples
Epoch 1/10
36095/36095 [==============================] - 47s 1ms/step - loss: 0.3925 - acc: 0.8188 - val_loss: 0.3360 - val_acc: 0.8537

Epoch 00001: val_acc improved from -inf to 0.85365, saving model to cnn.lenet.weights.best.hdf5
Epoch 2/10
36095/36095 [==============================] - 46s 1ms/step - loss: 0.2731 - acc: 0.8902 - val_loss: 0.3416 - val_acc: 0.8556

Epoch 00002: val_acc improved from 0.85365 to 0.85565, saving model to cnn.lenet.weights.best.hdf5
Epoch 3/10
36095/36095 [==============================] - 45s 1ms/step - loss: 0.2030 - acc: 0.9230 - val_loss: 0.3664 - val_acc: 0.8472

Epoch 00003: val_acc did not improve from 0.85565
[[1958  320]
 [ 301 1877]]
              precision    recall  f1-score   support

           0       0.87      0.86      0.86      2278
           1       0.85      0.86      0.86      2178

   micro avg       0.86      0.86      0.86      4456
   macro avg       0.86      0.86      0.86    

In [23]:
def CNN_alexnet(NUM_WORDS,n_dim,max_input_length,X_train, y_train,batch_size,epochs,X_val, y_val) :
  model = Sequential()
  model.add(Embedding(NUM_WORDS,n_dim, input_length=max_input_length))
  model.add(SpatialDropout1D(0.2))
  
  model.add(Conv1D(128, 3, activation='relu'))
  model.add(MaxPooling1D())
  model.add(BatchNormalization())
  
  model.add(Conv1D(256, 3, activation='relu'))
  model.add(MaxPooling1D())
  model.add(BatchNormalization())
  
  model.add(Conv1D(512, 3, activation='relu'))
  model.add(Conv1D(512, 3, activation='relu'))
  model.add(Conv1D(512, 3, activation='relu'))
  model.add(MaxPooling1D())
  model.add(BatchNormalization())
 
  model.add(Flatten())
  
  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  weight_file="cnn.alexnet.weights.best.hdf5"
  model_name ="cnn.alexnet.h5"
  
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(weight_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks_list)
  
  model.save(model_name)
 
  return model_name,weight_file

# model fitting and saving into drive

x, y = CNN_alexnet(NUM_WORDS,n_dim,max_input_length,X_train, y_train,batch_size,epochs,X_val, y_val)
model_evalute(x, y)

Train on 36095 samples, validate on 4011 samples
Epoch 1/10
36095/36095 [==============================] - 97s 3ms/step - loss: 2.5063 - acc: 0.6012 - val_loss: 0.7135 - val_acc: 0.6487

Epoch 00001: val_acc improved from -inf to 0.64872, saving model to cnn.alexnet.weights.best.hdf5
Epoch 2/10
36095/36095 [==============================] - 92s 3ms/step - loss: 0.5449 - acc: 0.7693 - val_loss: 0.5026 - val_acc: 0.7806

Epoch 00002: val_acc improved from 0.64872 to 0.78060, saving model to cnn.alexnet.weights.best.hdf5
Epoch 3/10
36095/36095 [==============================] - 92s 3ms/step - loss: 0.3923 - acc: 0.8293 - val_loss: 0.4443 - val_acc: 0.8000

Epoch 00003: val_acc improved from 0.78060 to 0.80005, saving model to cnn.alexnet.weights.best.hdf5
Epoch 4/10
36095/36095 [==============================] - 92s 3ms/step - loss: 0.3455 - acc: 0.8549 - val_loss: 0.4852 - val_acc: 0.7953

Epoch 00004: val_acc did not improve from 0.80005
Epoch 5/10
36095/36095 [=========================

In [26]:
def CNN_vggnet(NUM_WORDS,n_dim,max_input_length,X_train, y_train,batch_size,epochs,X_val, y_val) :
  model = Sequential()
  model.add(Embedding(NUM_WORDS,n_dim, input_length=max_input_length))
  model.add(SpatialDropout1D(0.2))
  
  model.add(Conv1D(128, 3, activation='relu'))
  model.add(Conv1D(128, 3, activation='relu'))
  model.add(MaxPooling1D())
  model.add(BatchNormalization())
  
  model.add(Conv1D(256, 2, activation='relu'))
  model.add(Conv1D(256, 2, activation='relu'))
  model.add(MaxPooling1D())
  model.add(BatchNormalization())
  
  model.add(Conv1D(384, 3, activation='relu'))
  model.add(Conv1D(384, 3, activation='relu'))
  model.add(Conv1D(384, 3, activation='relu'))
  model.add(MaxPooling1D())
  model.add(BatchNormalization())
  
  model.add(Conv1D(512, 3, activation='relu'))
  model.add(Conv1D(512, 3, activation='relu'))
  model.add(Conv1D(512, 3, activation='relu'))
  model.add(MaxPooling1D())
  model.add(BatchNormalization())
  
  model.add(Conv1D(512, 3, activation='relu'))
  model.add(Conv1D(512, 3, activation='relu'))
  model.add(Conv1D(512, 3, activation='relu'))
  model.add(MaxPooling1D())
  model.add(BatchNormalization())
  
  model.add(Flatten())
  
  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  weight_file="cnn.vggnet.weights.best.hdf5"
  model_name ="cnn.vggnet.h5"
  
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(weight_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks_list)
  
  model.save(model_name)
  
  return model_name,weight_file

# model fitting and saving into drive

x, y = CNN_vggnet(NUM_WORDS,n_dim,max_input_length,X_train, y_train,batch_size,epochs,X_val, y_val)
model_evalute(x, y)

Train on 36095 samples, validate on 4011 samples
Epoch 1/10
36095/36095 [==============================] - 124s 3ms/step - loss: 0.7744 - acc: 0.5906 - val_loss: 0.6918 - val_acc: 0.6756

Epoch 00001: val_acc improved from -inf to 0.67564, saving model to cnn.vggnet.weights.best.hdf5
Epoch 2/10
36095/36095 [==============================] - 117s 3ms/step - loss: 0.5041 - acc: 0.7853 - val_loss: 0.6418 - val_acc: 0.7524

Epoch 00002: val_acc improved from 0.67564 to 0.75243, saving model to cnn.vggnet.weights.best.hdf5
Epoch 3/10
36095/36095 [==============================] - 117s 3ms/step - loss: 0.3706 - acc: 0.8454 - val_loss: 0.4183 - val_acc: 0.8078

Epoch 00003: val_acc improved from 0.75243 to 0.80778, saving model to cnn.vggnet.weights.best.hdf5
Epoch 4/10
36095/36095 [==============================] - 117s 3ms/step - loss: 0.2932 - acc: 0.8802 - val_loss: 0.4404 - val_acc: 0.8045

Epoch 00004: val_acc did not improve from 0.80778
Epoch 5/10
36095/36095 [========================

In [27]:
def Simple_RNN(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val) :
  model = Sequential()
  model.add(Embedding(NUM_WORDS,n_dim, input_length=max_input_length))
  model.add(SpatialDropout1D(0.2))
  
  model.add(SimpleRNN(256))
  
  model.add(Dense(1, activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  weight_file="simplernn.weights.best.hdf5"
  model_name ="simplernn.h5"
  
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(weight_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks_list)
  
  model.save(model_name)
  
  return model_name,weight_file

# model fitting and saving into drive

x, y = Simple_RNN(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val)
model_evalute(x, y)

Train on 36095 samples, validate on 4011 samples
Epoch 1/10
36095/36095 [==============================] - 154s 4ms/step - loss: 0.5022 - acc: 0.7496 - val_loss: 0.3730 - val_acc: 0.8382

Epoch 00001: val_acc improved from -inf to 0.83819, saving model to simplernn.weights.best.hdf5
Epoch 2/10
36095/36095 [==============================] - 152s 4ms/step - loss: 0.3156 - acc: 0.8690 - val_loss: 0.4183 - val_acc: 0.8180

Epoch 00002: val_acc did not improve from 0.83819
Epoch 3/10
36095/36095 [==============================] - 151s 4ms/step - loss: 0.2758 - acc: 0.8890 - val_loss: 0.3741 - val_acc: 0.8404

Epoch 00003: val_acc improved from 0.83819 to 0.84044, saving model to simplernn.weights.best.hdf5
[[1935  343]
 [ 340 1838]]
              precision    recall  f1-score   support

           0       0.85      0.85      0.85      2278
           1       0.84      0.84      0.84      2178

   micro avg       0.85      0.85      0.85      4456
   macro avg       0.85      0.85      0.85 

In [30]:
def RNN_LSTM(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val) :
  model = Sequential()
  model.add(Embedding(NUM_WORDS,n_dim, input_length=max_input_length))
  model.add(SpatialDropout1D(0.2))
  
  model.add(LSTM(256, dropout=0.2))
  
  model.add(Dense(1, activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  weight_file="RNN_LSTM.weights.best.hdf5"
  model_name ="RNN_LSTM.h5"
  
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(weight_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks_list)
  
  model.save(model_name)
  
  return model_name,weight_file

# model fitting and saving into drive

x, y = RNN_LSTM(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val)
model_evalute(x, y)

Train on 36095 samples, validate on 4011 samples
Epoch 1/10
36095/36095 [==============================] - 565s 16ms/step - loss: 0.4117 - acc: 0.8148 - val_loss: 0.3460 - val_acc: 0.8524

Epoch 00001: val_acc improved from -inf to 0.85241, saving model to RNN_LSTM.weights.best.hdf5
Epoch 2/10
36095/36095 [==============================] - 559s 15ms/step - loss: 0.2998 - acc: 0.8753 - val_loss: 0.3404 - val_acc: 0.8546

Epoch 00002: val_acc improved from 0.85241 to 0.85465, saving model to RNN_LSTM.weights.best.hdf5
Epoch 3/10
36095/36095 [==============================] - 558s 15ms/step - loss: 0.2582 - acc: 0.8970 - val_loss: 0.3742 - val_acc: 0.8447

Epoch 00003: val_acc did not improve from 0.85465
Epoch 4/10
36095/36095 [==============================] - 560s 16ms/step - loss: 0.2340 - acc: 0.9076 - val_loss: 0.3767 - val_acc: 0.8372

Epoch 00004: val_acc did not improve from 0.85465
[[1918  360]
 [ 259 1919]]
              precision    recall  f1-score   support

           0    

In [0]:
def Stacked_LSTM(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val) :
  model = Sequential()
  model.add(Embedding(NUM_WORDS,n_dim, input_length=max_input_length))
  model.add(SpatialDropout1D(0.2))
  
  model.add(LSTM(256, dropout=0.2, return_sequences=True))
  
  model.add(LSTM(256, dropout=0.2))
  
  model.add(Dense(1, activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  weight_file="Stacked_LSTM.weights.best.hdf5"
  model_name ="Stacked_LSTM.h5"
  
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(weight_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks_list)
  
  model.save(model_name)
  
  return model_name,weight_file

# model fitting and saving into drive

x, y = Stacked_LSTM(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val)
model_evalute(x, y)

Train on 36095 samples, validate on 4011 samples
Epoch 1/10
36095/36095 [==============================] - 1256s 35ms/step - loss: 0.3985 - acc: 0.8185 - val_loss: 0.3541 - val_acc: 0.8467

Epoch 00001: val_acc improved from -inf to 0.84667, saving model to Stacked_LSTM.weights.best.hdf5
Epoch 2/10
36095/36095 [==============================] - 1254s 35ms/step - loss: 0.3222 - acc: 0.8618 - val_loss: 0.3638 - val_acc: 0.8449

Epoch 00002: val_acc did not improve from 0.84667
Epoch 3/10
32000/36095 [=========================>....] - ETA: 2:18 - loss: 0.2622 - acc: 0.8949

In [0]:
def Bidirectional_LSTM(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val) :
  model = Sequential()
  model.add(Embedding(NUM_WORDS,n_dim, input_length=max_input_length))
  model.add(SpatialDropout1D(0.2))
  
  model.add(Bidirectional(LSTM(256, dropout=0.2)))
  
  model.add(Dense(1, activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  weight_file="bidirectionl.LSTM.weights.best.hdf5"
  model_name ="bidirectionl.LSTM.h5"
  
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(weight_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks_list)
  
  model.save(model_name)
  
  return model_name,weight_file

# model fitting and saving into drive

x, y = Bidirectional_LSTM(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val)
model_evalute(x, y)

In [0]:
def Bidirectional_stacked_LSTM(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val) :
  model = Sequential()
  model.add(Embedding(NUM_WORDS,n_dim, input_length=max_input_length))
  model.add(SpatialDropout1D(0.2))
  
  model.add(Bidirectional(LSTM(256, dropout=0.2,return_sequences=True)))
  
  model.add(Bidirectional(LSTM(256, dropout=0.2)))
  
  model.add(Dense(1, activation='sigmoid'))
  
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  weight_file="Bidirectional_stacked_LSTM.weights.best.hdf5"
  model_name ="Bidirectional_stacked_LSTM.h5"
  
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(weight_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks_list)
  
  model.save(model_name)
  
  return model_name,weight_file

# model fitting and saving into drive

x, y = Bidirectional_stacked_LSTM(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val)
model_evalute(x, y)

In [0]:
from keras.models import Model
from keras.layers import Input, concatenate
def multi_cnn(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val) :
  
  input_layer = Input(shape=(max_review_length,), dtype='int16', name='input') # supports integers +/- 32.7k 
  
  embedding_layer = Embedding(NUM_WORDS,n_dim, input_length=max_input_length) (input_layer)
  drop_embed_layer = SpatialDropout1D(0.2, name='drop_embed')(embedding_layer)

  conv_1 = Conv1D(128, 3, activation='relu', name='conv_1')(drop_embed_layer)
  maxp_1 = GlobalMaxPooling1D(name='maxp_1')(conv_1)

  conv_2 = Conv1D(128, 3, activation='relu', name='conv_2')(drop_embed_layer)
  maxp_2 = GlobalMaxPooling1D(name='maxp_2')(conv_2)

  conv_3 = Conv1D(128, 3, activation='relu', name='conv_3')(drop_embed_layer)
  maxp_3 = GlobalMaxPooling1D(name='maxp_3')(conv_3)

  concat = concatenate([maxp_1, maxp_2, maxp_3])

  dense_layer = Dense(256, activation='relu', name='dense')(concat)
  drop_dense_layer = Dropout(0.2, name='drop_dense')(dense_layer)
  dense_2 = Dense(512, activation='relu', name='dense_2')(drop_dense_layer)
  dropout_2 = Dropout(0.2, name='drop_dense_2')(dense_2)

  predictions = Dense(1, activation='sigmoid', name='output')(dropout_2)

  model = Model(input_layer, predictions)

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  weight_file="multi_cnn.weights.best.hdf5"
  model_name ="multi_cnn.LSTM.h5"
  
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(weight_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val),callbacks=callbacks_list)
  
  model.save(model_name)
  
  return model_name,weight_file

# model fitting and saving into drive

x, y = multi_cnn(NUM_WORDS,n_dim,max_input_length,X_train,y_train,batch_size,epochs,X_val, y_val)
model_evalute(x, y)